In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Create a class which can take in value and if possible also do backpropagation throught the network

In [4]:
class Value:
    """Stores a single scalar value and its gradient"""
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self._children = set(_children)
        self._op = _op
        slef._backward = lambda:None
    
    def __add__(self, other):
        out = Value(self.data + other.data,(self,other),'+')
        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self,other), '*')
        return out

IndentationError: expected an indented block (989827151.py, line 1)